# Análise Exploratória: Incompatibilidades nos Dados

**Objetivo**: Identificar todos os casos de incompatibilidades nos dados de absentismo para discussão com RH.

**Abordagem**: Trabalhar ao nível dos **códigos originais** (não níveis agregados) para evitar mascarar ou criar incompatibilidades artificiais.

## Metodologia:

1. **Criar Matriz de Compatibilidade**: Matriz 40x40 com todos os pares de códigos
2. **Definir Regras Manualmente**: Exportar matriz para Excel e preencher "Compatível" ou "Incompatível"
3. **Testar Incompatibilidades**: Usar matriz para identificar todos os casos reais no dataset
4. **Exportar Resultados**: Excel com detalhes (Data, Login, Nome, Códigos) para discussão com RH

---

## 1. Carregamento de Dados

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Carregar códigos (mesma estrutura do notebook de apresentação)
print('Carregando códigos...')
codigos = pd.read_excel('códigos.xlsx', engine='openpyxl')
codigos.columns = ['codigo', 'descricao', 'nivel_1', 'nivel_2', 'nivel_3', 'nivel_4']

# Limpar espaços
for col in codigos.columns:
    if codigos[col].dtype == 'object':
        codigos[col] = codigos[col].str.strip()

print(f'✅ Códigos carregados: {len(codigos)} códigos')
print(f'   Níveis: N1={codigos["nivel_1"].nunique()}, N2={codigos["nivel_2"].nunique()}, N3={codigos["nivel_3"].nunique()}, N4={codigos["nivel_4"].nunique()}')

# Carregar dataset
print('\nCarregando dataset...')
df_raw = pd.read_csv('combined_data.csv', sep=',', parse_dates=['Data'])

# Merge com códigos para ter todos os níveis
df_raw = df_raw.merge(
    codigos[['codigo', 'nivel_1', 'nivel_2', 'nivel_3', 'nivel_4']], 
    left_on='segmento_processado_codigo', 
    right_on='codigo', 
    how='left'
)
df_raw.drop('codigo', axis=1, inplace=True)

print(f'\n✅ Dataset carregado: {len(df_raw):,} registros')
print(f'   Período: {df_raw["Data"].min().strftime("%Y-%m-%d")} a {df_raw["Data"].max().strftime("%Y-%m-%d")}')
print(f'   Colaboradores únicos: {df_raw["login_colaborador"].nunique():,}')
print(f'   Colunas com níveis: {[c for c in df_raw.columns if "nivel" in c]}')

---

## 2. Criação da Matriz de Compatibilidade entre Códigos

In [ ]:
# Obter lista de todos os códigos únicos
lista_codigos = sorted(codigos['codigo'].unique())
n_codigos = len(lista_codigos)

print('='*90)
print('📋 CRIAÇÃO DA MATRIZ DE COMPATIBILIDADE')
print('='*90)
print(f'\nTotal de códigos: {n_codigos}')
print(f'Tamanho da matriz: {n_codigos}x{n_codigos} = {n_codigos*n_codigos:,} combinações')
print(f'\n🔍 Códigos encontrados:')
for i, codigo in enumerate(lista_codigos, 1):
    descricao = codigos[codigos['codigo'] == codigo]['descricao'].values[0]
    print(f'   {i:2d}. {codigo:15s} - {descricao}')

# Criar matriz de compatibilidade (DataFrame)
# Por padrão, inicializar tudo como "Compatível"
# O utilizador depois edita no Excel para marcar "Incompatível"
matriz_compat = pd.DataFrame(
    'Compatível',  # Valor padrão
    index=lista_codigos,
    columns=lista_codigos
)

# REGRA: Um código consigo mesmo é sempre compatível (diagonal)
for codigo in lista_codigos:
    matriz_compat.loc[codigo, codigo] = 'Compatível'

print(f'\n✅ Matriz criada com valores padrão: "Compatível"')
print(f'\n💡 Próximo passo: Exportar para Excel e editar manualmente as incompatibilidades')
print(f'   Dica: A matriz é simétrica - se A é incompatível com B, então B é incompatível com A')

---

## 3. Exportação da Matriz para Excel

In [ ]:
# Exportar matriz para Excel
output_matriz = 'matriz_compatibilidade_codigos.xlsx'

with pd.ExcelWriter(output_matriz, engine='openpyxl') as writer:
    # Exportar matriz
    matriz_compat.to_excel(writer, sheet_name='Matriz_Compatibilidade', index=True)
    
    # Criar folha de referência com descrições dos códigos
    df_ref = codigos[['codigo', 'descricao', 'nivel_1', 'nivel_2', 'nivel_3', 'nivel_4']].copy()
    df_ref = df_ref.sort_values('codigo')
    df_ref.to_excel(writer, sheet_name='Referência_Códigos', index=False)
    
    # Criar folha de instruções
    instrucoes = pd.DataFrame({
        'Instruções': [
            '1. Na folha "Matriz_Compatibilidade", cada célula representa um par de códigos',
            '2. Valores possíveis: "Compatível" ou "Incompatível"',
            '3. Por padrão, todos os pares estão marcados como "Compatível"',
            '4. Editar células para marcar "Incompatível" onde apropriado',
            '',
            'EXEMPLOS DE INCOMPATIBILIDADES:',
            '- Férias + Falta Injustificada = Incompatível',
            '- Presença + Ausência = Incompatível',
            '- Baixa Médica + Trabalho Pago = Incompatível',
            '',
            'REGRAS:',
            '- A matriz é SIMÉTRICA: se A é incompatível com B, então B é incompatível com A',
            '- Um código consigo mesmo é sempre Compatível (diagonal)',
            '',
            '5. Após editar, guardar ficheiro',
            '6. Executar próxima secção do notebook para testar incompatibilidades',
            '',
            'NOTA: Use a folha "Referência_Códigos" para consultar descrições'
        ]
    })
    instrucoes.to_excel(writer, sheet_name='INSTRUÇÕES', index=False)

print(f'✅ Matriz exportada: {output_matriz}')
print(f'\n📋 Folhas criadas:')
print(f'   1. INSTRUÇÕES: Como preencher a matriz')
print(f'   2. Matriz_Compatibilidade: Matriz {n_codigos}x{n_codigos} para editar')
print(f'   3. Referência_Códigos: Lista de códigos com descrições')
print(f'\n🎯 PRÓXIMO PASSO:')
print(f'   1. Abrir ficheiro: {output_matriz}')
print(f'   2. Editar folha "Matriz_Compatibilidade"')
print(f'   3. Marcar "Incompatível" onde apropriado')
print(f'   4. Guardar ficheiro')
print(f'   5. Executar secção 4 deste notebook')
print(f'\n⚠️  IMPORTANTE: Não mudar o nome do ficheiro nem das folhas!')

---

## 4. Teste de Incompatibilidades (executar APÓS editar a matriz)

**⚠️ ATENÇÃO**: Só executar esta secção DEPOIS de editar e guardar `matriz_compatibilidade_codigos.xlsx`

In [ ]:
# Carregar matriz editada
print('Carregando matriz de compatibilidade editada...')
matriz_editada = pd.read_excel(
    'matriz_compatibilidade_codigos.xlsx',
    sheet_name='Matriz_Compatibilidade',
    index_col=0
)

# Contar incompatibilidades definidas
total_incompativeis = (matriz_editada == 'Incompatível').sum().sum()
# Dividir por 2 porque matriz é simétrica
total_incompativeis_unicos = total_incompativeis // 2

print(f'✅ Matriz carregada')
print(f'   Tamanho: {matriz_editada.shape[0]}x{matriz_editada.shape[1]}')
print(f'   Pares marcados como "Incompatível": {total_incompativeis} (ou {total_incompativeis_unicos} únicos)')

# Identificar dias com múltiplos registros
print('\nIdentificando dias com múltiplos registros...')
registros_por_dia = df_raw.groupby(['login_colaborador', 'Data']).size()
dias_duplicados = registros_por_dia[registros_por_dia > 1]

print(f'✅ Total de dias-colaborador únicos: {len(registros_por_dia):,}')
print(f'   Dias com MÚLTIPLOS registros: {len(dias_duplicados):,} ({len(dias_duplicados)/len(registros_por_dia)*100:.1f}%)')

# Filtrar apenas dias duplicados
dias_dup_list = dias_duplicados.index.tolist()
df_dup = df_raw.set_index(['login_colaborador', 'Data']).loc[dias_dup_list].reset_index()

print(f'   Dataset de dias duplicados: {len(df_dup):,} registros')

### 4.1. Teste de Todas as Incompatibilidades

In [ ]:
print('='*90)
print('🚨 TESTANDO INCOMPATIBILIDADES')
print('='*90)

# Dicionário para guardar resultados por par de códigos incompatíveis
resultados_incompatibilidades = {}

# Iterar por todos os dias duplicados
total_dias_testados = 0
total_incompatibilidades_encontradas = 0

for (login, data), count in dias_duplicados.items():
    total_dias_testados += 1
    
    # Obter códigos deste dia
    registros = df_dup[(df_dup['login_colaborador'] == login) & (df_dup['Data'] == data)]
    codigos_dia = registros['segmento_processado_codigo'].values
    nome = registros['nome_colaborador'].iloc[0]
    
    # Testar todos os pares de códigos
    for i, cod1 in enumerate(codigos_dia):
        for cod2 in codigos_dia[i+1:]:  # Evitar duplicatas e comparação consigo mesmo
            # Verificar na matriz se são incompatíveis
            if cod1 in matriz_editada.index and cod2 in matriz_editada.columns:
                compatibilidade = matriz_editada.loc[cod1, cod2]
                
                if compatibilidade == 'Incompatível':
                    total_incompatibilidades_encontradas += 1
                    
                    # Criar chave única para este par (ordenado alfabeticamente)
                    par = tuple(sorted([cod1, cod2]))
                    
                    # Adicionar ao dicionário de resultados
                    if par not in resultados_incompatibilidades:
                        resultados_incompatibilidades[par] = []
                    
                    resultados_incompatibilidades[par].append({
                        'Data': data,
                        'Login': login,
                        'Nome': nome,
                        'Codigo_1': cod1,
                        'Codigo_2': cod2
                    })

print(f'\n✅ Teste concluído')
print(f'   Dias testados: {total_dias_testados:,}')
print(f'   Incompatibilidades encontradas: {total_incompatibilidades_encontradas:,}')
print(f'   Pares de códigos incompatíveis encontrados: {len(resultados_incompatibilidades)}')

# Mostrar resumo dos pares incompatíveis encontrados
if len(resultados_incompatibilidades) > 0:
    print(f'\n📋 RESUMO DOS PARES INCOMPATÍVEIS ENCONTRADOS:')
    for par, casos in sorted(resultados_incompatibilidades.items(), key=lambda x: len(x[1]), reverse=True):
        cod1, cod2 = par
        desc1 = codigos[codigos['codigo'] == cod1]['descricao'].values[0]
        desc2 = codigos[codigos['codigo'] == cod2]['descricao'].values[0]
        print(f'\n   🔴 {cod1} + {cod2}: {len(casos)} casos')
        print(f'      {desc1}')
        print(f'      {desc2}')
else:
    print(f'\n✅ Nenhuma incompatibilidade encontrada nos dados!')

---

## 5. Exportação dos Resultados

In [ ]:
if len(resultados_incompatibilidades) == 0:
    print('⚠️ Nenhuma incompatibilidade encontrada. Nada para exportar.')
else:
    # Criar Excel com uma folha por par de códigos incompatíveis
    output_file = 'incompatibilidades_encontradas.xlsx'
    
    with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
        # Criar folha de resumo
        resumo_data = []
        for par, casos in sorted(resultados_incompatibilidades.items(), key=lambda x: len(x[1]), reverse=True):
            cod1, cod2 = par
            desc1 = codigos[codigos['codigo'] == cod1]['descricao'].values[0]
            desc2 = codigos[codigos['codigo'] == cod2]['descricao'].values[0]
            resumo_data.append({
                'Código 1': cod1,
                'Descrição 1': desc1,
                'Código 2': cod2,
                'Descrição 2': desc2,
                'Casos': len(casos),
                'Folha': f'{cod1}+{cod2}'
            })
        
        df_resumo = pd.DataFrame(resumo_data)
        df_resumo.to_excel(writer, sheet_name='RESUMO', index=False)
        
        # Criar uma folha por par incompatível
        for par, casos in resultados_incompatibilidades.items():
            cod1, cod2 = par
            
            # Criar DataFrame com os casos
            df_casos = pd.DataFrame(casos)
            
            # Adicionar descrições dos códigos
            desc1 = codigos[codigos['codigo'] == cod1]['descricao'].values[0]
            desc2 = codigos[codigos['codigo'] == cod2]['descricao'].values[0]
            df_casos['Descrição_1'] = desc1
            df_casos['Descrição_2'] = desc2
            
            # Reordenar colunas
            df_casos = df_casos[['Data', 'Login', 'Nome', 'Codigo_1', 'Descrição_1', 'Codigo_2', 'Descrição_2']]
            
            # Nome da folha (máximo 31 caracteres)
            sheet_name = f'{cod1}+{cod2}'[:31]
            
            df_casos.to_excel(writer, sheet_name=sheet_name, index=False)
    
    print(f'✅ Resultados exportados: {output_file}')
    print(f'\n📋 Estrutura do ficheiro:')
    print(f'   - Folha RESUMO: {len(resultados_incompatibilidades)} pares incompatíveis')
    print(f'   - {len(resultados_incompatibilidades)} folhas com detalhes de cada par')
    print(f'\n📊 Total de casos: {total_incompatibilidades_encontradas:,}')
    print(f'\n🎯 Próximo passo: Analisar com RH e decidir ação para cada tipo de incompatibilidade')

---

## Resumo do Processo

### Workflow Completo:

1. ✅ **Carregamento de Dados**: Códigos e dataset
2. ✅ **Criação da Matriz**: Matriz 40x40 com todos os pares de códigos
3. ✅ **Exportação da Matriz**: `matriz_compatibilidade_codigos.xlsx`
4. 📝 **Edição Manual**: Marcar pares "Incompatíveis" no Excel
5. 🔍 **Teste de Incompatibilidades**: Identificar todos os casos reais nos dados
6. 📊 **Exportação de Resultados**: `incompatibilidades_encontradas.xlsx`

### Vantagens desta Abordagem:

- ✅ **Trabalha ao nível dos códigos originais** (não agregados)
- ✅ **Evita incompatibilidades artificiais** causadas pela agregação em níveis
- ✅ **Flexível**: Fácil adicionar/remover regras editando a matriz
- ✅ **Auditável**: Matriz explícita de todas as regras de compatibilidade
- ✅ **Reutilizável**: Matriz pode ser usada em análises futuras